## Import the necessary packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from sklearn.metrics import (auc, roc_curve, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import f1_score
from skopt.space import Integer, Real, Categorical
from imblearn.over_sampling import SMOTE
from collections import Counter

## Import the data and split into Training and Testing sets

In [2]:
# Import the data as a data frame with the first column as the index
df = pd.read_csv('/scratch/by2372/Transcriptomics_Final_Project/ML/tf_filtered.tsv', sep='\t', index_col=0)

# Sets the values of the factors and the output types
values = df.columns.drop(['ID','type'])
X = np.array(df[values])
y = np.array(df['type'])

# Create the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5, stratify=y)

In [3]:
pd.Series(y_test).value_counts()

Normal_lung      8
Lung_Cancer      7
Breast_Cancer    5
Normal_Breast    4
Name: count, dtype: int64

In [4]:
pd.Series(y_train).value_counts()

Normal_lung      22
Lung_Cancer      20
Breast_Cancer    16
Normal_Breast    11
Name: count, dtype: int64

## Balance the training data only using the SMOTE overbalancing method

In [5]:
# Define the SMOTE object for data overbalancing
smote = SMOTE(random_state=4,k_neighbors = 5)

# Fit a new training data set using SMOTE
X_SMOTE, y_SMOTE = smote.fit_resample(X_train,y_train)
# Print out the distribution of the original training y and the new balanced training y
print("Original class distribution:", Counter(y_train))
print("Resampled class distribution:", Counter(y_SMOTE))

Original class distribution: Counter({'Normal_lung': 22, 'Lung_Cancer': 20, 'Breast_Cancer': 16, 'Normal_Breast': 11})
Resampled class distribution: Counter({'Normal_lung': 22, 'Normal_Breast': 22, 'Lung_Cancer': 22, 'Breast_Cancer': 22})


## Utilize BayesSearchCV to find the optimal hyperparameters for Logistic Regression

In [6]:
# Define a Base logistic Regression object
logreg_obj = LogisticRegression(
    penalty = 'l1',
    multi_class = 'multinomial',
    solver = 'saga',
    random_state = 5
)

# Define the search space using skopt's space definition
logreg_search_space = {
    'max_iter': Integer(10, 1000),
    'C': Real(1e-4, 100, prior='log-uniform')
}

# Define a Bayes Search object for hyperparameter tuning
logreg_bayes_search = BayesSearchCV(
    estimator = logreg_obj,
    search_spaces = logreg_search_space,
    scoring={'AUC': 'roc_auc_ovr', 'F1': 'f1_micro'}, #Both AUC and F1 score will be kept track of 
    refit='F1', # F1 score will be used to find the best iteration of hyperparameters
    n_iter=30, # The search algorithm will run for 20 iterations
    cv=5,
    random_state=5,
    n_jobs=5,
    verbose=1
)

logreg_bayes_search.fit(X_SMOTE,y_SMOTE)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/share/apps/anaconda3/2024.02/

BayesSearchCV(cv=5,
              estimator=LogisticRegression(multi_class='multinomial',
                                           penalty='l1', random_state=5,
                                           solver='saga'),
              n_iter=30, n_jobs=5, random_state=5, refit='F1',
              scoring={'AUC': 'roc_auc_ovr', 'F1': 'f1_micro'},
              search_spaces={'C': Real(low=0.0001, high=100, prior='log-uniform', transform='normalize'),
                             'max_iter': Integer(low=10, high=1000, prior='uniform', transform='normalize')},
              verbose=1)

In [7]:
# predicted probabilities of test set
y_logreg_probs = logreg_bayes_search.predict_proba(X_test)
y_logreg_pred = logreg_bayes_search.predict(X_test)

# Extract optimization history
logreg_opt_df = pd.DataFrame(logreg_bayes_search.cv_results_)

# Extract the best score and best parameters
best_score_logreg = logreg_bayes_search.best_score_
best_params_logreg = logreg_bayes_search.best_params_

# Compute F1 score (treating all classes equally)
f1_macro_logreg = f1_score(y_test, y_logreg_pred, average='macro')
f1_weighted_logreg = f1_score(y_test, y_logreg_pred, average='weighted')
f1_micro_logreg = f1_score(y_test,y_logreg_pred,average = 'micro')

In [8]:
# Displays the best F1 score out of all iterations of the optimization with training data
best_score_logreg

0.9104575163398693

In [9]:
# Displays the best hyperparameters that resulted in the best F1 score with cross validations of the training data
best_params_logreg

OrderedDict([('C', 0.024937887830984284), ('max_iter', 606)])

In [10]:
# Find the unique classifications
classifications = np.unique(y)

# binarize the classes
y_test_logreg_bin = label_binarize(y_test, classes=classifications)

fpr = dict()
tpr = dict()
roc_auc = dict()

# One vs. rest AUC calculation for multiclass data
for i in range(len(classifications)):
    this_class = classifications[i]
    fpr[this_class], tpr[this_class], _ = roc_curve(y_test_logreg_bin[:, i], y_logreg_probs[:, i])
    roc_auc[this_class] = auc(fpr[this_class], tpr[this_class])
    print(f"AUC for class {this_class}: {roc_auc[this_class]:.2f}")

# compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"],_=roc_curve(y_test_logreg_bin.ravel(),y_logreg_probs.ravel())
roc_auc["micro"] = auc(fpr["micro"],tpr["micro"])

AUC for class Breast_Cancer: 1.00
AUC for class Lung_Cancer: 0.98
AUC for class Normal_Breast: 1.00
AUC for class Normal_lung: 0.98


In [11]:
roc_auc["micro"]

0.9913194444444444

In [12]:
# Create a dictionary of metrics
metrics_dict = {
    'Best Parameters': [best_params_logreg],
    'Best Score (CV)': [best_score_logreg],
    'F1 Micro': [f1_micro_logreg],
    'AUC Micro': [roc_auc["micro"]],
    'AUC Breast Cancer': [roc_auc["Breast_Cancer"]],
    'AUC Lung Cancer': [roc_auc["Lung_Cancer"]],
    'AUC Normal Breast': [roc_auc["Normal_Breast"]],
    'AUC Normal Lung': [roc_auc["Normal_lung"]]
}

# Convert the dictionary into a single dataframe for exporting
metrics_df = pd.DataFrame.from_dict(metrics_dict, orient='index')

# Create path with csv name then export to csv
metrics_path = "/scratch/by2372/Transcriptomics_Final_Project/ML/Results/All_TFs_Logreg_metrics.csv"
metrics_df.to_csv(metrics_path, index = True)

## Logistic Regression Feature Selection With Best Hyperparameters

In [13]:
# Define the set parameters that were not optimized
best_params_logreg['penalty'] = 'l1'
best_params_logreg['multi_class'] = 'multinomial'
best_params_logreg['solver'] = 'saga'
best_params_logreg['random_state'] = 5

# Train the logistic regression model with the best parameters 
final_logreg = LogisticRegression(
    **best_params_logreg
)

# Fit the optimized model to the balanced data set
final_logreg.fit(X_SMOTE, y_SMOTE)

/share/apps/anaconda3/2024.02/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.024937887830984284, max_iter=606,
                   multi_class='multinomial', penalty='l1', random_state=5,
                   solver='saga')

In [14]:
# Get the features and class names
TFs = np.array(values)
class_names = final_logreg.classes_

class_coefs = dict()

# Loop through the coefficients of the fitted logreg model
for i, coefs in enumerate(final_logreg.coef_):
    class_name_i = class_names[i]  # Align class name with index
    # Define a dataframe that contains the whole list TFs with associated coefficients for classifying a specific class
    coef_df = pd.DataFrame({
        'TF': TFs,
        'Coefficient': coefs,
        'Class': class_name_i
    })
    # Sort the coefficients in descending order and append just the top 15 to the class_coefs dictionary
    coef_df = coef_df.sort_values(by = 'Coefficient', ascending = False)
    top_coef_df = coef_df.head(15)
    class_coefs[class_name_i] = top_coef_df

# Loop through each dictionary in class_coefs and export to desired directory
for key in class_coefs.keys():
    out_path = f"/scratch/by2372/Transcriptomics_Final_Project/ML/Results/{key}_Logreg_features_TFs.csv"
    this_df = class_coefs[key]
    this_df.to_csv(out_path, index = True)


## Utilize BayesSearchCV to find the optimal hyperparameters for Random Forest

In [15]:
# Define a Base logistic Regression object
rf_obj = RandomForestClassifier(
    bootstrap=True,
    random_state = 5
)

# Define the search space using skopt's space definition
rf_search_space = {
    'criterion': Categorical(['gini', 'entropy', 'log_loss']),    # Splitting criterion
    'n_estimators': Integer(10, 1000), # Number of trees
    'max_depth': Integer(3, 50), # Tree depth
    'max_features': Real(0.1, 1.0, prior='uniform'), # Fraction of features
    'min_samples_split': Integer(2, 20), # Min samples to split
    'min_samples_leaf': Integer(1, 20), # Min samples per leaf 
    'max_samples': Real(0.1, 1.0, prior='uniform')# Fraction of samples if bootstrap=True
}

# Define a Bayes Search object for hyperparameter tuning
rf_search = BayesSearchCV(
    estimator = rf_obj,
    search_spaces = rf_search_space,
    scoring={'AUC': 'roc_auc_ovr', 'F1': 'f1_micro'}, #Both AUC and F1 score will be kept track of 
    refit='F1', # F1 score will be used to find the best iteration of hyperparameters
    n_iter=30, # The search algorithm will run for 20 iterations
    cv=5,
    random_state=5,
    n_jobs=5,
    verbose=1
)

# Fit the search algorithm to the balanced training data
rf_search.fit(X_SMOTE,y_SMOTE)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

BayesSearchCV(cv=5, estimator=RandomForestClassifier(random_state=5), n_iter=30,
              n_jobs=5, random_state=5, refit='F1',
              scoring={'AUC': 'roc_auc_ovr', 'F1': 'f1_micro'},
              search_spaces={'criterion': Categorical(categories=('gini', 'entropy', 'log_loss'), prior=None),
                             'max_depth': Integer(low=3, high=50, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0...., transform='normalize'),
                             'max_samples': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=20, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=10, high=1000, prior='uniform', transform='normalize')},
              verbose=1)

In [16]:
# predicted probabilities of test set
y_rf_probs = rf_search.predict_proba(X_test)
y_rf_pred = rf_search.predict(X_test)

# Extract optimization history
rf_opt_df = pd.DataFrame(rf_search.cv_results_)

# Extract the best score and best parameters
best_score_rf = rf_search.best_score_
best_params_rf = rf_search.best_params_

# Compute F1 score (treating all classes equally)
f1_macro_rf = f1_score(y_test, y_rf_pred, average='macro')
f1_weighted_rf = f1_score(y_test, y_rf_pred, average='weighted')
f1_micro_rf = f1_score(y_test,y_rf_pred,average = 'micro')

In [17]:
# Displays the best F1 score out of all iterations of the optimization with training data
best_score_rf

0.9437908496732026

In [18]:
# Displays the best hyperparameters that resulted in the best F1 score with cross validations of the training data
best_params_rf

OrderedDict([('criterion', 'entropy'),
             ('max_depth', 50),
             ('max_features', 1.0),
             ('max_samples', 0.9355050593971552),
             ('min_samples_leaf', 4),
             ('min_samples_split', 2),
             ('n_estimators', 1000)])

In [19]:
# Find the unique classifications
classifications = np.unique(y)

# binarize the classes
y_test_rf_bin = label_binarize(y_test, classes=classifications)

fpr_rf = dict()
tpr_rf = dict()
roc_auc_rf = dict()

# One vs. rest AUC calculation for multiclass data
for i in range(len(classifications)):
    this_class = classifications[i]
    fpr_rf[this_class], tpr_rf[this_class], _ = roc_curve(y_test_rf_bin[:, i], y_rf_probs[:, i])
    roc_auc_rf[this_class] = auc(fpr_rf[this_class], tpr_rf[this_class])
    print(f"AUC for class {this_class}: {roc_auc_rf[this_class]:.2f}")

# compute micro-average ROC curve and ROC area
fpr_rf["micro"], tpr_rf["micro"],_=roc_curve(y_test_rf_bin.ravel(),y_rf_probs.ravel())
roc_auc_rf["micro"] = auc(fpr_rf["micro"],tpr_rf["micro"])

AUC for class Breast_Cancer: 1.00
AUC for class Lung_Cancer: 0.97
AUC for class Normal_Breast: 1.00
AUC for class Normal_lung: 0.98


In [20]:
roc_auc_rf["micro"]

0.9855324074074074

In [21]:
# Create a dictionary of metrics
metrics_dict_rf = {
    'Best Parameters': [best_params_rf],
    'Best Score (CV)': [best_score_rf],
    'F1 Micro': [f1_micro_rf],
    'AUC Micro': [roc_auc_rf["micro"]],
    'AUC Breast Cancer': [roc_auc_rf["Breast_Cancer"]],
    'AUC Lung Cancer': [roc_auc_rf["Lung_Cancer"]],
    'AUC Normal Breast': [roc_auc_rf["Normal_Breast"]],
    'AUC Normal Lung': [roc_auc_rf["Normal_lung"]]
}

# Convert the dictionary into a single dataframe for exporting
metrics_rf_df = pd.DataFrame.from_dict(metrics_dict_rf, orient='index')

# Create path with csv name then export to csv
metrics_path_rf = "/scratch/by2372/Transcriptomics_Final_Project/ML/Results/All_TFs_RF_metrics.csv"
metrics_rf_df.to_csv(metrics_path_rf, index = True)

## Random Forest Feature Selection With Best Hyperparameters

In [22]:
# Define the set parameters that were not optimized
best_params_rf['bootstrap'] = True
best_params_rf['random_state'] = 5

In [23]:
def rf_feature_selection(X_train,y_train,class_name,TF_names,best_params):
    
    y_train_new = [1 if x == class_name else 0 for x in y_train]

    # Train Random Forest
    rf = RandomForestClassifier(
            **best_params_rf
        )
    rf.fit(X_train,y_train_new)

    importances = rf.feature_importances_
    importances_df = pd.DataFrame({'Feature': TF_names, 'Importance': importances, 'Class': class_name})

    # Rank features by importance
    importances_df = importances_df.sort_values(by='Importance', ascending=False).reset_index()

    # Select top 15 features
    top_TFs = importances_df[:15]

    out_path = f"/scratch/by2372/Transcriptomics_Final_Project/ML/Results/{class_name}_Randfor_features_TFs.csv"
    top_TFs.to_csv(out_path, index = True)

In [24]:
for name in classifications:
    rf_feature_selection(X_SMOTE,y_SMOTE,name,TFs,best_params_rf)